In [20]:
from processor.processor import Processor as processor
from database.adatabase import ADatabase
from xgboost import XGBRegressor
from statistics import mean
import math
import pandas as pd
import numpy as np
from scipy.optimize import minimize
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from tqdm import tqdm
from dotenv import load_dotenv
from asset.stock import Stock
from asset.bond import Bond
from asset.option import Option
import warnings
import pytz
import copy
warnings.simplefilter(action="ignore")
import pickle

In [21]:
market = ADatabase("market")
sec = ADatabase("sec")
market = ADatabase("market")
fred = ADatabase("fred")
db = ADatabase("sapling")

In [22]:
fred.connect()
spy = fred.retrieve("sp500")
spy = spy.rename(columns={"value":"spy"})
spy["spy"] = spy["spy"].replace(".",np.nan)
spy.dropna(inplace=True)
spy["spy"] = [float(x) for x in spy["spy"]]
spy["date"] = spy["date"].shift(-5)
spy = processor.column_date_processing(spy)
fred.disconnect()

In [23]:
prices = []
market.connect()
sim = processor.column_date_processing(market.retrieve("prices")).merge(spy[["date","spy"]],on="date",how="left")
market.disconnect()

In [24]:
db.cloud_connect()
model_df = db.retrieve("model")
db.disconnect()


In [25]:
model = pickle.loads(model_df.iloc[0]["model"])

In [26]:
sim["prediction"] = model.predict(sim[["adjclose","spy"]])

In [27]:
sim

,date,adjclose,year,quarter,month,week,weekday,ticker,spy,prediction
0,2023-07-17,53.300,2023,3,7,29,0,LNT,4409.53,54.066162
1,2023-07-18,53.005,2023,3,7,29,1,LNT,4439.26,53.861248
2,2023-07-19,53.710,2023,3,7,29,2,LNT,4472.16,54.449463
3,2023-07-20,54.780,2023,3,7,29,3,LNT,4510.04,56.194138
4,2023-07-21,55.740,2023,3,7,29,4,LNT,4505.42,55.934513
...,...,...,...,...,...,...,...,...,...,...
125837,2024-07-09,174.240,2024,3,7,28,1,ZTS,5475.09,173.361404
125838,2024-07-10,173.490,2024,3,7,28,2,ZTS,5509.01,173.361404
125839,2024-07-11,177.740,2024,3,7,28,3,ZTS,5537.02,178.040604
125840,2024-07-12,179.040,2024,3,7,28,4,ZTS,5567.19,179.865799


In [28]:
db.cloud_connect()
db.drop("sim")
db.store("sim",sim[sim["date"]==sim["date"].max()])
db.disconnect() 

In [30]:
db.cloud_connect()
db.drop("backtest")
db.store("backtest",sim)
db.create_index("backtest","ticker")
db.disconnect() 